In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch

In [2]:
X = torch.randint(0,10,(100,10))
Y = X[:,0]

In [3]:
from metal.end_model import EndModel
from metal.modules import LSTMModule

input_size = 1
hidden_size = 10
vocab_size = 10

lstm_module = LSTMModule(input_size, hidden_size, vocab_size)
em = EndModel(seed=1, input_module=lstm_module)

ModuleNotFoundError: No module named 'end_model'

[autoreload of metal.end_model.em_defaults failed: Traceback (most recent call last):
  File "/Users/bradenjh/anaconda3/envs/metal/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/bradenjh/anaconda3/envs/metal/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 368, in superreload
    module = reload(module)
  File "/Users/bradenjh/anaconda3/envs/metal/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/Users/bradenjh/anaconda3/envs/metal/lib/python3.6/importlib/__init__.py", line 159, in reload
    name=parent_name) from None
ImportError: parent 'metal.end_model' not in sys.modules
]


TypeError: __init__() missing 1 required positional argument: 'input_dim'